In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install transformers datasets lora accelerate

In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

In [ ]:
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import json


# Carregar os dados
with open('/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/base-sintetica.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Preparar o dataset
formatted_data = [{"input_text": f"Pergunta: {item['question']} Contexto: ", "target_text": f"Resposta: {item['answer']}"} for item in data]
dataset = Dataset.from_list(formatted_data)
train_test_split = dataset.train_test_split(test_size=0.3)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

# Inicializar o tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Função de pré-processamento
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["target_text"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Aplicar pré-processamento aos datasets de treinamento e validação
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/260 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [ ]:
# Carregar o modelo GPT-2
model = GPT2LMHeadModel.from_pretrained("pierreguillou/gpt2-small-portuguese")

# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=25,
    weight_decay=0.001,
    save_total_limit=2,
    gradient_accumulation_steps=2
)

# Inicializar o treinador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

# Treinar o modelo
trainer.train()

# Salvar o modelo treinado e o tokenizer
trainer.save_model("/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/modelo")
tokenizer.save_pretrained("/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/modelo")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,0.767585
2,No log,0.562147
4,No log,0.529561
6,No log,0.511941
8,No log,0.501496
10,No log,0.494772
12,No log,0.488231
14,No log,0.485631
16,0.691000,0.487502
18,0.691000,0.482985


('/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/modelo/tokenizer_config.json',
 '/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/modelo/special_tokens_map.json',
 '/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/modelo/vocab.json',
 '/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/modelo/merges.txt',
 '/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/modelo/added_tokens.json')

In [ ]:
!pip install faiss-cpu sentence-transformers transformers

In [7]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import os

# Carregar o modelo de embeddings
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

# Função para extrair embeddings de texto
def get_embeddings(texts):
    embeddings = embedding_model.encode(texts, convert_to_tensor=True).cpu().numpy()
    return embeddings

# Carregar e indexar documentos
documents = []
document_path = "/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/textos_extraidos"
for filename in os.listdir(document_path):
    if filename.endswith(".txt"):
        with open(os.path.join(document_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            documents.append((content))

# Verificar se documentos foram carregados corretamente
print(f"Total de documentos carregados: {len(documents)}")

# Extrair embeddings dos documentos
doc_texts = [doc[1] for doc in documents]
doc_embeddings = get_embeddings(doc_texts)

# Verificar dimensões dos embeddings
print(f"Dimensões dos embeddings: {doc_embeddings.shape}")

# Criar o índice FAISS
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total de documentos carregados: 58
Dimensões dos embeddings: (58, 384)


In [8]:
from transformers import pipeline as hf_pipeline

# Carregar o modelo ajustado para geração de respostas
model_path = "/content/drive/MyDrive/Doutorado/NLP/atividades/tp3/modelo"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(model_path)

qa_pipeline = hf_pipeline("text-generation", model=model, tokenizer=tokenizer_gpt2)

def search_documents(query, k=5):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, k)
    valid_indices = [i for i in indices[0] if i < len(documents)]
    results = [documents[i] for i in valid_indices]
    return results

def generate_answer(question, retrieved_documents):
    context = "\n".join([doc[1] for doc in retrieved_documents[:3]])
    input_text = f"Resposta:"
    result = qa_pipeline(input_text, max_new_tokens=20, num_return_sequences=1)
    return result[0]['generated_text']

def ask_question(question):
    retrieved_docs = search_documents(question, k=5)
    if not retrieved_docs:
        return "Desculpe, não consegui encontrar informações relevantes."
    answer = generate_answer(question, retrieved_docs)
    return answer

# Exemplo de uso
questions = [
    "Qual é o objetivo principal da Lei nº 12.089/2009?",
    "Como deve ser realizada a matrícula em cursos a distância?",
    "Qual é a responsabilidade do Poder Público em relação à acessibilidade?",
    "Quais são os direitos dos estagiários?"
]

for question in questions:
    answer = ask_question(question)
    print(f"{question}")
    print(f"{answer}")
    print("="*50)

Qual é o objetivo principal da Lei nº 12.089/2009?
Resposta: Lei 5/ que assegura aos, por, de e, e de às às, por aos dos
Como deve ser realizada a matrícula em cursos a distância?
Resposta: Lei de criação nº. de forma de,, de lei de, de, de, do e
Qual é a responsabilidade do Poder Público em relação à acessibilidade?
Resposta: Lei.2010 não infra distância estabelece com obrigatória escolares cursos distância obrigatório e distância mínima de distância escolar com
Quais são os direitos dos estagiários?
Resposta: solicitação realizada 9 anos sem para com com matrícula.0 com matriculados matrícula cancelada no, repro


In [9]:
# Verificar se os embeddings são válidos
print("Verificação de embeddings...")
for i, embedding in enumerate(doc_embeddings):
    if not np.all(np.isfinite(embedding)):
        print(f"Embedding inválido no documento {i}")

# Testar a busca e geração de respostas com exemplos
questions = [
    "Qual é a política de matrícula tardia?",
    "Como funciona o trancamento de curso?",
    "Quais são as normas para transferência de curso?",
    "O que acontece se eu perder o prazo de matrícula?",
    "Quais são as regras para troca de turno?"
]

for question in questions:
    answer = ask_question(question)
    print(f"Pergunta: {question}")
    print(f"{answer}")
    print("="*50)


Verificação de embeddings...
Pergunta: Qual é a política de matrícula tardia?
Resposta: mínima de 10 anos ) de no a pelo um. pelo de em por do à. ao do
Pergunta: Como funciona o trancamento de curso?
Resposta: solicitação- autorizado para pelo solicitação., realizada a em de autorizado, para parecer, parecer com parecer
Pergunta: Quais são as normas para transferência de curso?
Resposta: Lei/2008 e compatível com a/ de a legislação Lei de ou lei de Lei de. outra
Pergunta: O que acontece se eu perder o prazo de matrícula?
Resposta: Lei. 2// que lei/ deve é ser Lei/ pelo pelo, aprovado pela de pelo
Pergunta: Quais são as regras para troca de turno?
Resposta: objetivos- a incluem em atividades de o comunidade de a o atividades, pesquisa, extensão de pesquisa e
